In [ ]:
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'

In [ ]:
from pathlib import Path
from tempfile import TemporaryDirectory

import harmony


environments = {
    'http://localhost:3000': harmony.Environment.LOCAL,
    'https://harmony.sit.earthdata.nasa.gov': harmony.Environment.SIT,
    'https://harmony.uat.earthdata.nasa.gov': harmony.Environment.UAT,
    'https://harmony.earthdata.nasa.gov': harmony.Environment.PROD,
}
assert harmony_host_url in environments
harmony_client = harmony.Client(env=environments[harmony_host_url])

request = harmony.Request(
    collection=harmony.Collection(id='OPERA_L2_RTC-S1_V1'),
    granule_name=[
        'OPERA_L2_RTC-S1_T035-073251-IW2_20240512T020817Z_20240512T122756Z_S1A_30_v1.0'
    ],
    format='image/png',
)
job_id = harmony_client.submit(request)
harmony_client.wait_for_processing(job_id)

with TemporaryDirectory() as temp_dir:
    output_files = [
        Path(future.result())
        for future in harmony_client.download_all(job_id, directory=temp_dir)
    ]

    assert len(output_files) == 3
    assert output_files[0].suffixes == ['.png']
    assert output_files[1].suffixes == ['.pgw']
    assert output_files[2].suffixes == ['.png', '.aux', '.xml']

    reference_data = Path('reference_data')
    assert output_files[0].read_bytes() == (reference_data / 'rgb.png').read_bytes()
    assert output_files[1].read_bytes() == (reference_data / 'rgb.pgw').read_bytes()
    assert (
        output_files[2].read_bytes()
        == (reference_data / 'rgb.png.aux.xml').read_bytes()
    )